In [ ]:
import os
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=gpu,floatX=float32'

In [ ]:
import theano
import theano.tensor as T
import theano.tensor.nnet as NN
import numpy as NP
import numpy.random as RNG
import h5py
import dataUtils
from collections import OrderedDict
from Wrapper import Model, rmsprop
%matplotlib inline
import matplotlib.pyplot as plt
from theano.tensor.signal import downsample

Base CNN

In [ ]:
def get_conv_out_dim(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    output_dim = ((img_row - filter_row) / stride_row + 1) * ((img_col - filter_col) / stride_col + 1) * nr_filters * 1
    return output_dim

def get_conv_out_shape(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    return (((img_row - filter_row) / stride_row + 1), ((img_col - filter_col) / stride_col + 1))

def _step(cur_in):
    
    maxpool_shape=(2, 2)
    
    conv1_shape = (20, 1, 5, 5, 1, 1)
    conv1_h = model.conv(cur_in = cur_in, name = 'conv1', shape=conv1_shape)
    pool1 = downsample.max_pool_2d(conv1_h, maxpool_shape, ignore_border = True)
    conv1_act = T.tanh(pool1)
    
    conv2_shape = (50, 20, 5, 5, 1, 1)
    conv2_h = model.conv(cur_in = conv1_act, name = 'conv2', shape=conv2_shape)
    pool2 = downsample.max_pool_2d(conv2_h, maxpool_shape, ignore_border = True)
    conv2_act = T.tanh(pool2)
    
    fc1 = NN.sigmoid(model.fc(cur_in = conv2_act.flatten(2), name = 'fc1', shape=(24200, out_dim)))
    #fc1 = NN.softmax(model.fc(cur_in = conv2_act.flatten(2), name = 'fc1', shape=(24200, out_dim)))
    return fc1
    

img_shape = (100, 100)
out_dim = 10
model = Model()
img_in = T.tensor4()
seg_img = T.tensor4()
img_tar = T.matrix()
img_out = _step(img_in)
_EPSI = 1e-6
MAX_V = 20
cost = T.mean(T.sum(NN.binary_crossentropy(T.clip(img_out, _EPSI, 1.0 - _EPSI), img_tar), axis=1))
#cost = T.mean(NN.categorical_crossentropy(T.clip(img_out, _EPSI, 1.0 - _EPSI), img_tar))
cost_sum_sal = 0.0
cost_sal = [None]*10
for i in xrange(10):
    sal_img = T.grad(T.sum(img_out[:,i]), img_in)
    sal_img = sal_img / (T.sum(sal_img)+_EPSI)
    cost_sal[i] = T.sum((sal_img-seg_img[:,i:i+1])**2)
    cost_sal[i] = T.switch(cost_sal[i]>MAX_V, MAX_V, cost_sal[i])
    cost_sum_sal += cost_sal[i]

#grad_func = theano.function([img_in, img_tar], rms_grad.values()[4], allow_input_downcast=True)    
#test_func = theano.function([img_in, img_tar, seg_img], cost, allow_input_downcast=True)
test_base_conv_func = theano.function([img_in, img_tar, seg_img], [cost, img_out, cost_sum_sal], allow_input_downcast=True)
train_base_conv_func = theano.function([img_in, img_tar], [cost, img_out], updates=rmsprop(cost, model.weightsPack.getW_list(), lr=1e-4, rho=0.9, epsilon=1e-8), allow_input_downcast=True)
model_base_conv = model
#predict_func = theano.function([img_in], img_out, allow_input_downcast=True)

In [ ]:
train_batch_size = 64
test_batch_size = 64
test_his = []

from dataUtils import BouncingMNIST
image_size = 100
bmnist = BouncingMNIST(train_batch_size, image_size, 'train/inputs', 'train/targets', with_clutters=False, clutter_size_max = 14, scale_range = 0.1)
bmnist_test = BouncingMNIST(test_batch_size, image_size, 'test/inputs', 'test/targets', with_clutters=False, clutter_size_max = 14, scale_range = 0.1)
for i in xrange(50):
    for j in xrange(200):
        #data, label, seg=bmnist.GetStaticBatch(num_digits=RNG.randint(5))
        data, label, seg=bmnist.GetStaticBatch(num_digits=1)
        n_cost, net_out = train_base_conv_func(data, label)
        #acc = NP.mean(NP.argmax(net_out, axis=1)==NP.argmax(label, axis=1))
        acc = NP.mean(NP.sum(((net_out>0.5)==(label>0.5))==(label>0.5), axis=1))
        print 'Epoch = ', str(i), ' Batch = ', str(j), ' Cost = ', n_cost, ' ACC = ', acc
    test_acc = []
    test_sal_cost = []
    for j in xrange(len(bmnist_test.data_)/test_batch_size):
        data, label, seg=bmnist_test.GetStaticBatch(num_digits=1)
        n_cost, net_out, n_sal_cost = test_base_conv_func(data, label, seg)
        test_sal_cost.append(n_sal_cost)
        test_acc.append(NP.mean(NP.sum(((net_out>0.5)==(label>0.5))==(label>0.5), axis=1)))
        #test_acc = NP.mean(NP.argmax(net_out, axis=1)==NP.argmax(label, axis=1))
    print 'Epoch = ', str(i), ' Batch = ', str(j), ' Test ACC = ', NP.mean(NP.asarray(test_acc)), 'Test Sal Cost = ', NP.mean(NP.asarray(test_sal_cost))
    test_his.append(NP.mean(NP.asarray(test_acc)))

Fully conv

In [ ]:
def get_conv_out_dim(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    output_dim = ((img_row - filter_row) / stride_row + 1) * ((img_col - filter_col) / stride_col + 1) * nr_filters * 1
    return output_dim

def get_conv_out_shape(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    return (((img_row - filter_row) / stride_row + 1), ((img_col - filter_col) / stride_col + 1))

def _step(cur_in):
    
    conv1_shape = (20, 1, 5, 5, 2, 2)
    
    conv1_h = model.conv(cur_in = cur_in, name = 'conv1', shape=conv1_shape)
    conv1_out_shape = get_conv_out_shape(img_shape, conv1_shape)
    conv1_h = T.tanh(conv1_h)
    
    conv2_shape = (50, 20, 5, 5, 2, 2)
    conv2_h = model.conv(cur_in = conv1_h, name = 'conv2', shape=conv2_shape)
    conv2_h = T.tanh(conv2_h)
    
    conv3_shape = (50, 50, 5, 5, 2, 2)
    conv3_h = model.conv(cur_in = conv2_h, name = 'conv3', shape=conv3_shape)
    conv3_h = T.tanh(conv3_h)
    
    fc1 = NN.sigmoid(model.fc(cur_in = conv3_h.flatten(2), name = 'fc1', shape=(4050, 10)))
    #fc2 = NN.softmax(model.fc(cur_in = fc1, name = 'fc2', shape=(100, out_dim)))
    return fc1
    

img_shape = (100, 100)
out_dim = 10
model = Model()
img_in = T.tensor4()
seg_img = T.tensor4()
img_tar = T.matrix()
img_out = _step(img_in)
_EPSI = 1e-6
cost = T.mean(T.sum(NN.binary_crossentropy(T.clip(img_out, _EPSI, 1.0 - _EPSI), img_tar), axis=1))
#cost = T.mean(NN.categorical_crossentropy(T.clip(img_out, _EPSI, 1.0 - _EPSI), img_tar))
MAX_V = 20
cost_sum_sal = 0.0
cost_sal = [None]*10
for i in xrange(10):
    sal_img = T.grad(T.sum(img_out[:,i]), img_in)
    sal_img = sal_img / (T.sum(sal_img)+_EPSI)
    cost_sal[i] = T.sum((sal_img-seg_img[:,i:i+1])**2)
    cost_sal[i] = T.switch(cost_sal[i]>MAX_V, MAX_V, cost_sal[i])
    cost_sum_sal += cost_sal[i]
    
#grad_func = theano.function([img_in, img_tar], rms_grad.values()[4], allow_input_downcast=True)    
#test_func = theano.function([img_in, img_tar, seg_img], cost, allow_input_downcast=True)
test_fc_func = theano.function([img_in, img_tar, seg_img], [cost, img_out, cost_sum_sal], allow_input_downcast=True)
train_fc_func = theano.function([img_in, img_tar], [cost, img_out], updates=rmsprop(cost, model.weightsPack.getW_list(), lr=1e-4, rho=0.9, epsilon=1e-8), allow_input_downcast=True)
model_fc = model
#predict_func = theano.function([img_in], img_out, allow_input_downcast=True)

In [ ]:
train_batch_size = 64
test_batch_size = 64
test_fc_his = []

from dataUtils import BouncingMNIST
image_size = 100
bmnist = BouncingMNIST(train_batch_size, image_size, 'train/inputs', 'train/targets', with_clutters=False, clutter_size_max = 14, scale_range = 0.1)
bmnist_test = BouncingMNIST(test_batch_size, image_size, 'test/inputs', 'test/targets', with_clutters=False, clutter_size_max = 14, scale_range = 0.1)
for i in xrange(50):
    for j in xrange(5):
        #data, label, seg=bmnist.GetStaticBatch(num_digits=RNG.randint(5))
        data, label, seg=bmnist.GetStaticBatch(num_digits=1)
        n_cost, net_out = train_fc_func(data, label)
        #acc = NP.mean(NP.argmax(net_out, axis=1)==NP.argmax(label, axis=1))
        acc = 1.0*NP.sum((net_out+label>1.5))/NP.sum((net_out+label>0.5))
        print 'Epoch = ', str(i), ' Batch = ', str(j), ' Cost = ', n_cost, ' ACC = ', acc
    test_acc = []
    test_sal_cost = []
    for j in xrange(len(bmnist_test.data_)/test_batch_size):
        data, label, seg=bmnist_test.GetStaticBatch(num_digits=1)
        n_cost, net_out, n_sal_cost = test_fc_func(data, label, seg)
        test_sal_cost.append(n_sal_cost)
        test_acc.append(NP.mean(NP.sum(((net_out>0.5)==(label>0.5))==(label>0.5), axis=1)))
        #test_acc = NP.mean(NP.argmax(net_out, axis=1)==NP.argmax(label, axis=1))
    print 'Epoch = ', str(i), ' Batch = ', str(j), ' Test ACC = ', NP.mean(NP.asarray(test_acc)), 'Test Sal Cost = ', NP.mean(NP.asarray(test_sal_cost))
    test_fc_his.append(NP.mean(NP.asarray(test_acc)))
    

In [ ]:
len(bmnist_test.data_)

Sal Fully Conv

In [ ]:
def get_conv_out_dim(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    output_dim = ((img_row - filter_row) / stride_row + 1) * ((img_col - filter_col) / stride_col + 1) * nr_filters * 1
    return output_dim

def get_conv_out_shape(img_shape, conv_shape):
    img_row, img_col = img_shape
    nr_filters, nr_channels, filter_row, filter_col, stride_row, stride_col = conv_shape
    return (((img_row - filter_row) / stride_row + 1), ((img_col - filter_col) / stride_col + 1))

def _step(cur_in):
    
    conv1_shape = (20, 1, 5, 5, 2, 2)
    
    conv1_h = model.conv(cur_in = cur_in, name = 'conv1', shape=conv1_shape)
    conv1_out_shape = get_conv_out_shape(img_shape, conv1_shape)
    conv1_h = T.tanh(conv1_h)
    
    conv2_shape = (50, 20, 5, 5, 2, 2)
    conv2_h = model.conv(cur_in = conv1_h, name = 'conv2', shape=conv2_shape)
    conv2_h = T.tanh(conv2_h)
    
    fc1 = NN.sigmoid(model.fc(cur_in = conv2_h.flatten(2), name = 'fc1', shape=(24200, 10)))
    #fc2 = NN.softmax(model.fc(cur_in = fc1, name = 'fc2', shape=(100, out_dim)))
    return fc1
    

img_shape = (100, 100)
out_dim = 10
model = Model()
img_in = T.tensor4()
seg_img = T.tensor4()
img_tar = T.matrix()
img_out = _step(img_in)
_EPSI = 1e-6
cost = T.mean(T.sum(NN.binary_crossentropy(T.clip(img_out, _EPSI, 1.0 - _EPSI), img_tar), axis=1))
#cost = T.mean(NN.categorical_crossentropy(T.clip(img_out, _EPSI, 1.0 - _EPSI), img_tar))
MAX_V = 20
cost_sum_sal = 0.0
cost_sal = [None]*10
for i in xrange(10):
    sal_img = T.grad(T.sum(img_out[:,i]), img_in)
    sal_img = sal_img / (T.sum(sal_img)+_EPSI)
    cost_sal[i] = T.sum((sal_img-seg_img[:,i:i+1])**2)
    cost_sal[i] = T.switch(cost_sal[i]>MAX_V, MAX_V, cost_sal[i])
    cost_sum_sal += cost_sal[i]
    
#grad_func = theano.function([img_in, img_tar], rms_grad.values()[4], allow_input_downcast=True)    
#test_func = theano.function([img_in, img_tar, seg_img], cost, allow_input_downcast=True)
test_sal_fc_func = theano.function([img_in, img_tar, seg_img], [cost, img_out, cost_sum_sal], allow_input_downcast=True)
train_sal_fc_func = theano.function([img_in, img_tar, seg_img], [cost, img_out, cost_sum_sal], updates=rmsprop(cost+cost_sum_sal, model.weightsPack.getW_list(), lr=1e-4, rho=0.9, epsilon=1e-8), allow_input_downcast=True)
model_sal_fc = model
#predict_func = theano.function([img_in], img_out, allow_input_downcast=True)

In [ ]:
train_batch_size = 64
test_batch_size = 64
test_sal_fc_his = []

from dataUtils import BouncingMNIST
image_size = 100
bmnist = BouncingMNIST(train_batch_size, image_size, 'train/inputs', 'train/targets', with_clutters=False, clutter_size_max = 14, scale_range = 0.1)
bmnist_test = BouncingMNIST(test_batch_size, image_size, 'test/inputs', 'test/targets', with_clutters=False, clutter_size_max = 14, scale_range = 0.1)
for i in xrange(50):
    for j in xrange(200):
        #data, label, seg=bmnist.GetStaticBatch(num_digits=RNG.randint(5))
        data, label, seg=bmnist.GetStaticBatch(num_digits=1)
        n_cost, net_out, n_cost_sal = train_sal_fc_func(data, label, seg)
        #acc = NP.mean(NP.argmax(net_out, axis=1)==NP.argmax(label, axis=1))
        acc = NP.sum((net_out+label>1.5))/NP.sum((net_out+label>0.5))
        print 'Epoch = ', str(i), ' Batch = ', str(j), ' Cost = ', n_cost, ' Cost Sal = ', n_cost_sal, ' ACC = ', acc
    
    test_acc = []
    test_sal_cost = []
    for j in xrange(len(bmnist_test.data_)/test_batch_size):
        data, label, seg=bmnist_test.GetStaticBatch(num_digits=1)
        n_cost, net_out, n_sal_cost = test_sal_fc_func(data, label, seg)
        test_sal_cost.append(n_sal_cost)
        test_acc.append(NP.mean(NP.sum(((net_out>0.5)==(label>0.5))==(label>0.5), axis=1)))
        #test_acc = NP.mean(NP.argmax(net_out, axis=1)==NP.argmax(label, axis=1))
    print 'Epoch = ', str(i), ' Batch = ', str(j), ' Test ACC = ', NP.mean(NP.asarray(test_acc)), 'Test Sal Cost = ', NP.mean(NP.asarray(test_sal_cost))
    test_sal_fc_his.append(NP.mean(NP.asarray(test_acc)))

In [ ]:
batch_size = 200
image_size = 30
bmnist_test = BouncingMNIST(batch_size, image_size, 'train/inputs', 'train/targets', with_clutters=False, clutter_size_max = 14, scale_range = 0.1)

In [ ]:
model.weightsPack.idxs

In [ ]:
model.weightsPack.vect[4].get_value()

In [ ]:
model.weightsPack.vect[4].get_value()

In [ ]:
data, label, seg=bmnist_test.GetStaticBatch(num_digits=1)
n_cost_cls, n_cost, net_out = train_func(data, label, seg)
acc = NP.mean(NP.argmax(net_out, axis=1)==NP.argmax(label, axis=1))
print ' Cost CLS = ', n_cost_cls, ' Cost SUM = ', n_cost, ' ACC = ', acc

In [ ]:
data, label, seg=bmnist.GetStaticBatch(num_digits=1)
print label
print NP.sum(grad_func(data, label))

In [ ]:
label[4]

In [ ]:
np.mean(seg[4][6])

In [ ]:
data, label, seg=bmnist.GetStaticBatch(num_digits=1)
plt.imshow(data[4][0])
plt.show()
plt.imshow(seg[4][7])
plt.show()

In [ ]:
''' 
load data 
'''
in_dim = 49
out_dim = 49
batch_size = 16

X_train, Y_train=dataUtils.read_txt('ptb.train.txt')
#X_test, Y_test=dataUtils.read_txt('ptb.test.txt')


In [ ]:
def _step(trash, cur_in, rec_in1, rec_in2, rec_in3):
    
    gru1_h = model.gru(cur_in = cur_in, rec_in = rec_in1, name='gru1', shape=(in_dim, 20))
    gru2_h = model.gru(cur_in = gru1_h, rec_in = rec_in2, name='gru2', shape=(20, 20))
    gru3_h = model.gru(cur_in = gru2_h, rec_in = rec_in3, name='gru3', shape=(20, 20))
    merge = T.concatenate([gru1_h, gru2_h, gru3_h], axis=1)
    fc1 = model.fc(cur_in = merge, name='fc1', shape=(60, out_dim))
    return [fc1] + [gru1_h, gru2_h, gru3_h]
    
model = Model()
char_in = T.tensor3()
char_tar = T.tensor3()
_char_in = char_in.dimshuffle(1, 0, 2)
starts = T.matrix()

sc, _ = theano.scan(_step, sequences=[_char_in], outputs_info=[starts, T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20))])
char_out = sc[0].dimshuffle(1, 0, 2)
_EPSI = 1e-6
cost = T.mean(T.sum(NN.binary_crossentropy(T.clip(char_out, _EPSI, 1.0 - _EPSI), char_tar), axis=2))

#cost = T.mean(T.sum((char_tar - char_out) ** 2, axis=2))
train_func = theano.function([char_in, starts, char_tar], [cost, char_out], updates=rmsprop(cost, model.weightsPack.getW_list()), allow_input_downcast=True)
predict_func = theano.function([char_in, starts], char_out, allow_input_downcast=True)

In [ ]:
def _step(cur_in, trash, rec_in1, rec_in2, rec_in3):
    
    gru1_h = model2.gru(cur_in = cur_in, rec_in = rec_in1, name='gru1', shape=(in_dim, 100))
    gru2_h = model2.gru(cur_in = gru1_h, rec_in = rec_in2, name='gru2', shape=(100, 100))
    gru3_h = model2.gru(cur_in = gru2_h, rec_in = rec_in3, name='gru3', shape=(100, 100))
    merge = T.concatenate([gru1_h, gru2_h, gru3_h], axis=1)
    fc1 = model.fc(cur_in = merge, name='fc1', shape=(60, out_dim))
    return [fc1] + [gru1_h, gru2_h, gru3_h]
    
model2 = Model()


char_in = T.tensor3()
char_tar = T.tensor3()
_char_in = char_in.dimshuffle(1, 0, 2)
starts = T.matrix()

sc, _ = theano.scan(_step, sequences=[_char_in], outputs_info=[starts, T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20)), T.zeros((char_in.shape[0], 20))])
char_out = sc[0].dimshuffle(1, 0, 2)

#cost = T.mean(T.sum((char_tar - char_out) ** 2, axis=2))

#train_func = theano.function([char_in, starts, char_tar], [cost, char_out], updates=rmsprop(cost, model.weightsPack.getW_list()), allow_input_downcast=True)
#Jpredict_func = theano.function([char_in, starts], char_out, allow_input_downcast=True)
jacobi = T.jacobian(char_out[0, :50, 0:3].flatten(), char_in)
jacobi_func = theano.function([char_in, starts], jacobi, allow_input_downcast=True)

In [ ]:
model2.weightsPack = model.weightsPack
model2.layersPack = model.layersPack

In [ ]:
str(0.5555)

In [ ]:
pb = ProgressBar(40000, title='Epoch '+str(1), key='outer')
for i in pb:
    pb_inner = ProgressBar(1, title='Inner', key='inner')

In [ ]:
len(X_train)

In [ ]:

batch_size = 64
pb = ProgressBar(len(X_train), title='Epoch '+str(i), key='outer')
for i in xrange(10):
    print 'epoch ', i

    for j in xrange(len(X_train)/batch_size):
        train_cost, _ = train_func(X_train[j*batch_size:(j+1)*batch_size], X_train[j*batch_size:(j+1)*batch_size,0], Y_train[j*batch_size:(j+1)*batch_size]) 
        pb_inner = ProgressBar(1, title='Inner', key='inner')

In [ ]:
J_predict = predict_func(X_train[0:1], X_train[0:1, 0])
J_in = NP.concatenate((X_train[0:1, 0:1, :], J_predict[0:1, :-1, :]), axis=1)
v_jacobi = jacobi_func(X_train[0:1], X_train[0:1, 0])

In [ ]:
print J_predict[0, 3]
print Y_train[0, 3]

In [ ]:
print J_predict[0, 3]
print Y_train[0, 3]

In [ ]:
p_jacobi = jacobi_func(X_train[0:1], X_train[0:1, 0])
#print p_jacobi

In [ ]:
NP.shape(v_jacobi)

In [ ]:
NP.sum(abs(v_jacobi[30]), axis=2)

In [ ]:
help(ProgressBar)

In [ ]:
pb = ProgressBar(1000, title='Outer', key='outer')
for i in pb:
    pb_inner = ProgressBar(1, title='Inner', key='inner')
    print 'hhh'